# Interact with Deepseek-R1 via Ollama Python

In this notebook we will learn how to

1. List open models installed on your local machine
2. Stream responses from deepseek-r1
3. Track conversation history.

> Ollama python package can be installed from [PyPI](https://pypi.org/project/ollama/). I have provided a [conda environment](./environment.yml) file that will install it for you.

## Imports

In [ ]:
import ollama
from ollama import chat

## Open models installed on local machine

In [ ]:
def installed_models():
    '''
    Iterate through ollama models and return names as list
    '''
    return [md.model for md in ollama.list().models]

In [ ]:
local_models = installed_models()
local_models

## Stream Chat Responses from a model

In [ ]:
def get_stream(model_name: str, prompt_msg: str):
    '''get a streaming chat from a model
    '''
    stream = chat(
        model=model_name,
        messages=[{'role': 'user', 'content': prompt_msg}],
        stream=True,
    )

    return stream

In [ ]:
prompt_1 = """
Given five positive integers in a list, find the minimum and maximum values that
can be calculated by summing exactly four of the five integers. 
Print the respective minimum and maximum values. 
Code the solution in Python as a python function that accepts a Python list 
as a parameter.

Example

input  = [9, 3, 5, 7, 1]
Output = 16, 24
"""

In [ ]:
# use coder first.
stream = get_stream(local_models[0], prompt_1)

In [ ]:
response_fname = 'response_1.txt'
with open(response_fname, 'w') as writer:
    for chunk in stream:
      print(chunk['message']['content'], end='', flush=True)
      writer.write(chunk['message']['content'])

## Conversation history

> **Acknowledgement**: The format shown below is taken from the `ollama-python` [github](https://github.com/ollama/ollama-python/blob/main/examples/chat-with-history.py)

```python
history = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
  {
    'role': 'assistant',
    'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight.",
  },
  {
    'role': 'user',
    'content': 'What is the weather in Tokyo?',
  },
  {
    'role': 'assistant',
    'content': '...',
  },
]
```

In [ ]:
def get_stream(model_name: str, chat_history: list):
    '''get a streaming chat from a model
    '''
    stream = chat(
        model=model_name,
        messages=chat_history,
        stream=True,
    )

    return stream

In [ ]:
def format_message(history: list, role: str, content: str):
    '''Helper function for updating the message history
    '''
    prompt = {
        'role': role,
        'content': content
    }
    history.append(prompt)
    return history

In [ ]:
def read_response(file_name: str):
    '''Read a stored response from file
    '''
    response = ""
    with open(file_name, 'r') as reader:
        for line in reader:
            response += line.rstrip('\n')
    return response

In [ ]:
messages = []

In [ ]:
messages = format_message(messages, 'user', prompt_1)
messages

In [ ]:
stored_response = read_response(response_fname)
messages = format_message(messages, 'assistant', stored_response)

In [ ]:
prompt_2 = "split the solution into two functions for the max and min sums respectively."
messages = format_message(messages, 'user', prompt_2)
stream = get_stream(local_models[0], messages)

In [ ]:
response_fname = 'response_2.txt'
with open(response_fname, 'w') as writer:
    for chunk in stream:
      print(chunk['message']['content'], end='', flush=True)
      writer.write(chunk['message']['content'])